In [79]:
import utils
import math
import re
import itertools
from typing import NamedTuple
from collections import defaultdict, deque
import itertools

import matplotlib.pyplot as plt

## Day 13: Shuttle Search

[#](https://adventofcode.com/2020/day/13). Figure out a bus timetable. Buses have an ID number which indicates how often a bus leaves for the airport.

A bus with ID 5 departs from port every 5 minutes, starting at `t=0`. Every bus leaves the port at time 0 then repeats.

In [135]:
test13 = """939
7,13,x,x,59,x,31,19
""".splitlines()

inp13 = utils.get_input(13, splitlines=True)

def parse_13(inp=test13, verbose=False):
    return int(inp[0]), [int(i) for i in inp[1].split(",") if i != "x"]

earliest, bus_ids = parse_13()
print(earliest, bus_ids)

939 [7, 13, 59, 31, 19]


I'm doing this the simple brute force way, though there is a simple way to mathify all this too!

In [36]:
def solve_13(inp=test13):
    earliest, bus_ids = parse_13(inp)
    
    for time in range(earliest, earliest + max(bus_ids)):
        for n in bus_ids:
            if time %n == 0:
                time_waited = time - earliest
                return time_waited * n
            
    
assert solve_13() == 295
solve_13(inp13)

4722

## Part 2

So now we have staggered departure times, which makes all this a bit more complex.

We need to find the earliest times at which all the buses leave at offsets matching their positions in the list.

Add a bus tuple to make it easier to read:

In [167]:
test_13a = "17,x,13,19"
test_13b = "67,x,7,59,61"
test_13d = "1789,37,47,1889"

class Bus(NamedTuple):
    every: int
    offset: int
        
def parse_13b(inp=test_13a):
    """returns a list of tuples of bus_id, offset"""
    inp = inp.split(",")
    return [Bus(int(i), int(inp.index(i))) for i in inp if i!="x"]

inp = parse_13b()
inp

[Bus(every=17, offset=0), Bus(every=13, offset=2), Bus(every=19, offset=3)]

In [170]:
def solve_13b(inp=test_13a, start=10**10, end=10**16):
    """a simple brute force solution"""
    inp = parse_13b(inp)
    cycle = inp[0].every
    for t in range(cycle*start, end, cycle):
        h = [(t+bus.offset) % bus.every == 0 for bus in inp]
        if all(h):
            print(f"At time {t:,} buses are staggered by their offsets")
            return t
        
assert solve_13b(start=0, end=5000) == 3417
assert solve_13b(test_13b, start=10, end=10**11) == 779210
assert solve_13b(test_13d, start=10**5, end=10**11) == 1202161486

At time 3,417 buses are staggered by their offsets
At time 779,210 buses are staggered by their offsets
At time 1,202,161,486 buses are staggered by their offsets


The brute force solution works for the test inputs but is too slow for the actual input. So to speed it up I'm going to try and solve this in pairs, which should quickly lead to bigger timejumps.

In [190]:
def solve_13_pair(inp, cycle, start=1, end=10**16):

    for t in range(cycle*start, end, cycle):
        h = [(t+bus.offset) % bus.every == 0 for bus in inp]
        if all(h):
            #print(f"At time {t:,} buses are staggered by their offsets")
            return t

In [191]:
inp = parse_13b(test_13a)
first_bus = inp[0]
cycle = inp[0].every

c = []

for bus in inp[1:]:
    cycle = solve_13_pair([first_bus, bus], cycle)
    print(cycle, bus)
    c.append(cycle)

102 Bus(every=13, offset=2)
510 Bus(every=19, offset=3)


In [195]:
math.prod(c)

52020